In [3]:
import open3d as o3d
import numpy as np
import glob
import copy
import math

voxel_size = 0.035
max_correspondence_dist_coarse = voxel_size * 15
max_correspondence_dist_fine = voxel_size * 1.5
radius_normal = voxel_size * 2
radius_feature = voxel_size * 5

def preprocess_point_cloud(pcd, voxel_size):
    #print(":: Downsample with a voxel size %.3f." % voxel_size)
    #pcd_down = o3d.geometry.voxel_down_sample(pcd, voxel_size)
    radius_normal = voxel_size * 2
    #print(":: Estimate normal with search radius %.3f." % radius_normal)
    o3d.geometry.estimate_normals(pcd,o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
    radius_feature = voxel_size * 5
    #print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.registration.compute_fpfh_feature(pcd,o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_fpfh


def load_point_clouds(directory ,voxel_size ,n,g):
    pcds = []
    fpfh = []
    for i in range(n):
        pcd = o3d.io.read_point_cloud(directory + "%d.ply" % i)
        #pcd_down = o3d.geometry.voxel_down_sample(pcd, voxel_size=voxel_size)
        pcd_down = pcd
        '''
        ss = np.asarray(pcd_down.points)
        ss = ss - np.mean(ss)
        pcd_down.points = o3d.utility.Vector3dVector(ss)
        '''
        pcd_fpfh = preprocess_point_cloud(pcd_down, voxel_size)
        fpfh.append(pcd_fpfh)
        pcds.append(pcd_down)
    return pcds, fpfh

def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    voxel_size = 0.035
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, distance_threshold,
        o3d.registration.TransformationEstimationPointToPoint(False), 30, 
        [o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.5),
         o3d.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)],
        o3d.registration.RANSACConvergenceCriteria(4000000, 500))
    return result

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

def pairwise_registration(source, target, source_fpfh, target_fpfh, voxel_size):
    #source_s, source_fpfh = preprocess_point_cloud(source, voxel_size)
    #target_t, target_fpfh = preprocess_point_cloud(target, voxel_size)
    
    '''
    distance_threshold = voxel_size * 3
    result_ransac = o3d.registration.registration_ransac_based_on_correspondence(source, target, 
                                                                                 o3d.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold),
                                                                                 max_correspondence_dist_fine,
                                                                                 o3d.registration.TransformationEstimationPointToPlane())
    
    trM = np.asarray([[1.0,0.0,0.0,0.0],[0.0,1.0,0.0,0.0],[0.0,0.0,1.0,0.0],[0.0,0.0,0.0,1.0]])
    '''

    result_ransac = execute_global_registration(source, target,source_fpfh, target_fpfh, voxel_size)
    icp_fine = o3d.registration.registration_icp(source, target, max_correspondence_dist_fine,
                                                 result_ransac.transformation,
                                                 o3d.registration.TransformationEstimationPointToPlane())
    transformation_icp = icp_fine.transformation
    #transformation_icp = result_ransac.transformation
    information_icp = o3d.registration.get_information_matrix_from_point_clouds(source, target, 
                                                                                max_correspondence_dist_fine,
                                                                                icp_fine.transformation)
    return transformation_icp, information_icp


def main_registration(pcds,fpfh, max_correspondence_dist_coarse,max_correspondence_dist_fine, voxel_size):
    pose_graph = o3d.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
            target_id = (source_id + 1) % n_pcds
            print("performing transformation for:")
            print(source_id)
            print(target_id)
            transformation_icp, information_icp = pairwise_registration(pcds[source_id], pcds[target_id], 
                                                                        fpfh[source_id], fpfh[target_id],
                                                                        voxel_size)
            draw_registration_result(pcds[source_id], pcds[target_id], transformation_icp)
            print("Build o3d.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(o3d.registration.PoseGraphNode(np.linalg.inv(odometry)))
                pose_graph.edges.append(o3d.registration.PoseGraphEdge(source_id,target_id,transformation_icp,
                                                                       information_icp,uncertain=False))
    
            else:  # loop closure case
                pose_graph.edges.append(o3d.registration.PoseGraphEdge(source_id,target_id,transformation_icp,
                                                                       information_icp,uncertain=True))
    return pose_graph

def final_fun(dirtry, g):
    #Directory of Point-cloud goes here
    directory = dirtry
    n = len(glob.glob1(directory,"*.ply"))
    o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)
    
    pcds_down, pcd_fpfh = load_point_clouds(directory,voxel_size,n,g)
    o3d.visualization.draw_geometries(pcds_down)
    n_pcds = len(pcds_down)
    
    #check if there is any scalling between the two point clouds
    ############################################################
    
    #Implementing Full Registration
    print("Full registration ...")
    pose_graph = main_registration(pcds_down, pcd_fpfh, max_correspondence_dist_coarse,max_correspondence_dist_fine, voxel_size)

    #Implementing Global Optimization
    print("Optimizing PoseGraph ...")
    option = o3d.registration.GlobalOptimizationOption(max_correspondence_distance=max_correspondence_dist_fine,
                                                       edge_prune_threshold=0.25,reference_node=0)
    o3d.registration.global_optimization(
        pose_graph, o3d.registration.GlobalOptimizationLevenbergMarquardt(),
        o3d.registration.GlobalOptimizationConvergenceCriteria(), option)

    print("Transform points and display")
    pcd_combined = o3d.geometry.PointCloud()
    for point_id in range(len(pcds_down)):
        print(point_id)
        print(pose_graph.nodes[point_id].pose)
        pcds_down[point_id].transform(pose_graph.nodes[point_id].pose)
        pcd_combined += pcds_down[point_id]
    o3d.visualization.draw_geometries(pcds_down)
    pcd_combined_down = o3d.geometry.voxel_down_sample(pcd_combined,voxel_size=voxel_size)
    #print(pcd_combined_down)
    o3d.io.write_point_cloud(dirtry + "multiway_registration.ply", pcd_combined)
    #o3d.io.write_point_cloud("C:/Users/Kathan/Desktop/ACLAB/3D_Registeration/DATA/lattest_newM/testing_main_copy/final.ply")
    #o3d.visualization.draw_geometries([pcd_combined])

In [6]:
import easydict
if __name__ == "__main__":
    n = input("How many Pcds do you have?")
    ang = input("Do you know their Angles? Y Or N")
    args = easydict.EasyDict({
                           "dir": "C:/Users/Kathan/Desktop/ACLAB/3D_Registeration/DATA/manne_top/iteration3/tests/test_raw_body/",
                           "n": int(n),
                           "angle" : ang 
                         })
    g = []    
    if args.angle == "yes" or args.angle == "y":
        g = []
        for i in range(1,args.n):
            print("Enter angle for pointcloud", i)
            h = input()  
            g.append(h)
    final_fun(args.dir, g)

Full registration ...
performing transformation for:
0
1
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.035,
   we use a liberal distance threshold 0.053.
Build o3d.registration.PoseGraph
performing transformation for:
1
2
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.035,
   we use a liberal distance threshold 0.053.
Build o3d.registration.PoseGraph
performing transformation for:
2
3
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.035,
   we use a liberal distance threshold 0.053.
Build o3d.registration.PoseGraph
performing transformation for:
3
4
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.035,
   we use a liberal distance threshold 0.053.
Build o3d.registration.PoseGraph
performing transformation for:
4
5
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is